In [1]:
from pymongo import MongoClient
from lxml import html

import os
import requests
import time
import random

mongo_connection_uri = os.environ.get("ME_CONFIG_MONGODB_URL")
client = MongoClient(mongo_connection_uri)

db = client["atcoder"]
submissions_raw_collection = db["submissions_raw"]
submission_raw_collection = db["submission_raw"]

In [2]:
def get_html(submission_url):
    url = f"https://atcoder.jp{submission_url}"
    cookies = {
        "_ga": "GA1.1.1669735030.1727884004",
        "REVEL_FLASH": "",
        "timeDelta": "2007",
        "REVEL_SESSION": "8ed0689f1af1fa89bab4edd30927ff7273e1ad0e-%00csrf_token%3AiecE%2FCD%2BWkZctrj%2BaFUUE0TovINExsqeelroJ6VJx9Q%3D%00%00_TS%3A1746190748%00%00UserName%3Adarcy_zhang%00%00a%3Afalse%00%00w%3Afalse%00%00SessionKey%3Af712625f246d5b11787eeeac74b6cefd3f43ebfeb6e05e5a6b9568a08ba4367cebdfbc32acea8a-0b0fdeaa544c89f44b7326075ad509e6ba62c1c71fa969e5d0cab3c4e61ee30d%00%00UserScreenName%3Adarcy_zhang%00",
        "_ga_RC512FD18N": "GS1.1.1730638744.10.1.1730638745.0.0.0",
    }

    headers = {
        "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
        "accept-language": "en-US,en;q=0.9,zh-CN;q=0.8,zh;q=0.7,en-GB;q=0.6,zh-TW;q=0.5",
        "cache-control": "no-cache",
        "pragma": "no-cache",
        "priority": "u=0, i",
        "referer": "https://atcoder.jp/contests/abc378/submissions?f.Task=abc378_a&f.LanguageName=Python&f.Status=AC&f.User=",
        "sec-ch-ua": '"Chromium";v="130", "Microsoft Edge";v="130", "Not?A_Brand";v="99"',
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": '"Windows"',
        "sec-fetch-dest": "document",
        "sec-fetch-mode": "navigate",
        "sec-fetch-site": "same-origin",
        "sec-fetch-user": "?1",
        "upgrade-insecure-requests": "1",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36 Edg/130.0.0.0",
    }

    response = requests.get(
        url,
        cookies=cookies,
        headers=headers,
    )
    response.encoding = "utf-8"
    return response.text

In [3]:
error_string = "500 Internal Server Error"
result = submissions_raw_collection.delete_many({"raw_html": {"$regex": error_string}})
print(f"Deleted {result.deleted_count} documents containing '{error_string}' in raw_html.")

submission_url_list = []

for submissions_raw_item in submissions_raw_collection.find():
    contest_id = submissions_raw_item["contest_id"]
    task_id = submissions_raw_item["task_id"]
    tree = html.fromstring(submissions_raw_item["raw_html"])
    url_list = tree.xpath("//a[@class='submission-details-link']")
    for url in url_list:
        submission_url = url.get("href")
        submission_url_list.append(
            {
                "contest_id": contest_id,
                "task_id": task_id,
                "submission_url": submission_url,
            }
        )
random.shuffle(submission_url_list)
print(len(submission_url_list))

Deleted 142 documents containing '500 Internal Server Error' in raw_html.
63029


In [4]:
def exec_task(submission_url_item):
    submission_url = submission_url_item["submission_url"]
    existing_submission = submission_raw_collection.find_one(
        {"submission_url": submission_url}
    )
    if existing_submission is None:
        time.sleep(0.3)
        raw_html = get_html(submission_url)
        submission_raw_collection.insert_one(
            {
                "contest_id": submission_url_item["contest_id"],
                "task_id": submission_url_item["task_id"],
                "submission_url": submission_url,
                "raw_html": raw_html,
            }
        )
    else:
        print(f"Submission with {submission_url} already exists, skipping insertion")

task_list = submission_url_list
while len(task_list) > 0:
    index = random.randrange(len(task_list))
    item = task_list.pop(index)
    try:
        exec_task(item)
    except Exception as e:
        print(e)
        task_list.append(item)

Submission with /contests/abc304/submissions/58846259 already exists, skipping insertion
Submission with /contests/abc281/submissions/53298909 already exists, skipping insertion
Submission with /contests/abc238/submissions/59400731 already exists, skipping insertion
Submission with /contests/abc322/submissions/59316505 already exists, skipping insertion
Submission with /contests/abc326/submissions/59073734 already exists, skipping insertion
Submission with /contests/abc350/submissions/59331339 already exists, skipping insertion
Submission with /contests/abc340/submissions/59323453 already exists, skipping insertion
Submission with /contests/abc314/submissions/57249291 already exists, skipping insertion
Submission with /contests/abc277/submissions/39734365 already exists, skipping insertion
Submission with /contests/ahc020/submissions/55040509 already exists, skipping insertion
Submission with /contests/abc270/submissions/59322464 already exists, skipping insertion
Submission with /cont